In [1]:
import numpy as np
import torch as t
#import torchvision.transforms.functional as TF
#import torchvision
#import torchvision.transforms as transforms
#import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = t.device("cuda" if t.cuda.is_available() else "cpu") 

# Parameters

In [18]:
C_in = 3
D_out = 441
epochs = 1 #how many epochs 
lr = 5e-5 #learning rate
bn = 1 #how many batches
bs = 1 #how many images in a batch
modelname = 'CNet'
lossb = np.zeros([2, 2, epochs])
sc = 100
nh = 5*sc
nw = 9*sc
t.manual_seed(0)

In [19]:
_, lossb[0,:,:] = multit()

RuntimeError: CUDA out of memory. Tried to allocate 1.62 GiB (GPU 0; 11.00 GiB total capacity; 8.53 GiB already allocated; 967.38 MiB free; 8.55 GiB reserved in total by PyTorch)

# Functions

In [5]:
def getbatch(bcr):
    label = t.rand([bs, D_out]).to(device)
    data = t.rand([bs, C_in, nh, nw]).to(device)
    return(data,label)

In [6]:
def getsize(szi,ker,srd): 
    pad = (0,0)
    dil = np.asarray((0,0))
    return(tuple((np.squeeze((szi+2*np.asarray(pad)-dil*[np.asarray(ker)-1]+1)/np.asarray(srd))).astype(int)))

In [20]:
class CNet(t.nn.Module):
    def __init__(self):
        super().__init__()
        cker = (5,5)
        csrd = (1,1)
        mker = (2,2)
        msrd = (2,2)
        self.conv0 = nn.Conv2d(C_in, 8, cker)
        self.conv1 = nn.Conv2d(8, 16, cker)
        self.pool = nn.MaxPool2d(mker)
        sz = getsize((nh,nw),cker,csrd)
        sz = getsize(sz,mker,msrd)
        sz = getsize(sz,cker,csrd)
        sz = getsize(sz,mker,msrd)
        sz = np.asarray(sz)-4-int((sc/2)%2)
        print(sz)
        self.linear0 = t.nn.Linear(16*122*222, 1000)
        self.linear1 = t.nn.Linear(1000, D_out)
    def forward(self, x):
        x = self.pool(F.relu(self.conv0(x)))
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.linear0(x))
        x = F.relu(self.linear1(x))
        return x
nnarchitectures = {'CNet' : CNet}
def multit():
    model = nnarchitectures[modelname]()
    model.to(device)
    optimizer = t.optim.Adam(model.parameters(), lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    lossa = np.zeros([2, epochs])
    xval, xtest = [t.zeros([bs, C_in, nh, nw]).to(device)]*2
    yval, ytest = [t.zeros([bs, D_out]).to(device)]*2
    [xtest, ytest] = getbatch(0)
    [xval, yval] = getbatch(1)
    for j in range(epochs):
        for i in range(bn):
            [xtrain, ytrain] = getbatch(i+2)
            try:
                del loss
            except:
                1
            criterion = t.nn.MSELoss(reduction='mean')
            #xtrain = xtrain.detach() #xtrain contains images of the circle
            y_pred = model(xtrain) #y_pred consists of coordinates of the circle center and its radius
            # Forward pass: Compute predicted y by passing x to the model
            #calculating loss between predicted parameters and parameters which were used for generation
            loss = criterion(y_pred, ytrain)
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        lossa[0][j] = loss.item()
        lossa[1][j] = criterion(model(xval), yval).item()
        del loss
        print('.', end='')
    return(model, lossa)

In [21]:
_, lossb[0,:,:] = multit()

[122 222]


RuntimeError: CUDA out of memory. Tried to allocate 1.62 GiB (GPU 0; 11.00 GiB total capacity; 8.53 GiB already allocated; 967.38 MiB free; 8.55 GiB reserved in total by PyTorch)